In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from pathlib import Path
from lmfit import Parameters
from lmfit import minimize, fit_report
from sklearn import metrics
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

In [ ]:
import pyroll.core as pr
import pyroll.pillar_model
import pyroll.neutral_point_estimator
import pyroll.sparling_spreading
import pyroll.hensel_force_torque
import pyroll.freiberg_flow_stress
import pyroll.ring_model_thermal
from pyroll.freiberg_flow_stress import flow_stress

In [ ]:
ROLL_SURFACE_TEMPERATURE = 50 + 273.15
DISK_ELEMENT_COUNT = 30

In [ ]:
pyroll.pillar_model.Config.PILLAR_COUNT = 30
pyroll.neutral_point_estimator.Config.ESTIMATOR = "EQUAL"
ROLL_SURFACE_TEMPERATURE = 50 + 273.15
DISK_ELEMENT_COUNT = 30

root_dir = Path.cwd()
result_dir = root_dir / "res"

In [ ]:
from rolling_sequence import finishing_pass_design_1, finishing_pass_design_2, finishing_pass_design_3, \
    finishing_pass_design_4
from in_profile import create_in_profile_for_finishing_train
from utilities import *

In [ ]:
@pr.CoolingPipe.DiskElement.heat_transfer_coefficient
def heat_transfer_coefficient_model_moreales(self: pr.CoolingPipe.DiskElement):
    mean_surface_temperature = (self.in_profile.surface_temperature + self.out_profile.surface_temperature) / 2
    profile_surface_temperature_celsius = mean_surface_temperature - 273.15
    return 13950 + 5.12 * 1e6 / profile_surface_temperature_celsius

In [ ]:
in_profile = create_in_profile_for_finishing_train(950 + 273.15)
in_profile.ring_temperatures = in_profile.ring_temperatures

In [ ]:
rolling_train_rpd1 = finishing_pass_design_1(ROLL_SURFACE_TEMPERATURE, DISK_ELEMENT_COUNT)
rolling_train_rpd2 = finishing_pass_design_2(ROLL_SURFACE_TEMPERATURE, DISK_ELEMENT_COUNT)
rolling_train_rpd3 = finishing_pass_design_3(ROLL_SURFACE_TEMPERATURE, DISK_ELEMENT_COUNT)
rolling_train_rpd4 = finishing_pass_design_4(ROLL_SURFACE_TEMPERATURE, DISK_ELEMENT_COUNT)

In [ ]:
rolling_train_rpd1.solve(in_profile)
rolling_train_rpd2.solve(in_profile)
rolling_train_rpd3.solve(in_profile)
#rolling_train_rpd4.solve(in_profile)

In [ ]:
sequences = [
    {"label": "RPD1", "sequence": rolling_train_rpd1},
    {"label": "RPD2", "sequence": rolling_train_rpd2},
    {"label": "RPD3", "sequence": rolling_train_rpd3},
    #{"label": "RPD4", "sequence": pass_design_3}
]

In [ ]:
def combine_simulated_and_measured_data():
    _data = []

    for entry in sequences:
        groove_label = entry["label"]
        sequence = entry["sequence"]
        roll_passes = sequence.roll_passes
        measurements = extract_wear_contours_from_measurement(groove_label)

        for measurement_id, labels, tonnages, measured_wear_contours in measurements:
            for roll_pass in roll_passes:
                for label, tonnage, measured_wear_contour in zip(labels, tonnages, measured_wear_contours):
                    if roll_pass.label == label:
                        wear_area = calculate_area_between_contours(roll_pass.roll.contour_line, measured_wear_contour)
                        normed_wear_area = wear_area / roll_pass.roll.groove.cross_section.area
                        mean_temperature = (roll_pass.in_profile.temperature + roll_pass.out_profile.temperature) / 2
                        if label in ["17-H", "21-H", "23-H", "27-H"]:
                            groove_type = "oval"
                        else:
                            groove_type = "round"
                        _data.append({
                            "groove": groove_label,
                            "measurement_id": measurement_id,
                            "label": label,
                            "groove_type": groove_type,
                            "tonnage": tonnage,
                            "wear_area": wear_area,
                            "normed_wear_area": normed_wear_area,
                            "temperature": mean_temperature,
                            "p_deformation_resistance": np.mean(pillar_deformation_resistance(roll_pass)),
                            "roll_force": roll_pass.roll_force
                        })

    return _data

In [ ]:
data = combine_simulated_and_measured_data()

In [ ]:
tonnages = []
normed_wear_areas = []
temperatures = []
deformation_resistances = []
types = []
for data_per_pass in data:
    normed_wear_areas.append(data_per_pass['normed_wear_area'])
    tonnages.append(data_per_pass['tonnage'])
    temperatures.append(data_per_pass['temperature'])
    deformation_resistances.append(data_per_pass['p_deformation_resistance'])
    types.append(data_per_pass['groove_type'])



In [ ]:
df = pd.DataFrame({
    'tonnage': np.array(tonnages),
    'normed_wear_area': np.array(normed_wear_areas),
    'temperature': np.array(temperatures),
    'deformation_resistance': np.array(deformation_resistances),
    'groove_type': np.array(types)
})

In [ ]:
features_for_clustering = ['tonnage', 'normed_wear_area', 'deformation_resistance', 'temperature']
fitting_array = df[features_for_clustering].values

In [ ]:
distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
K = range(1, 10)
for k in K:
    kmeanModel = KMeans(n_clusters=k, random_state=42).fit(fitting_array)
    df['cluster'] = kmeanModel.predict(fitting_array)

    distortions.append(sum(np.min(cdist(fitting_array, kmeanModel.cluster_centers_, 'euclidean'), axis=1) ** 2) / fitting_array.shape[0])
    inertias.append(kmeanModel.inertia_)

    mapping1[k] = distortions[-1]
    mapping2[k] = inertias[-1]

In [ ]:
fig, ax = plt.subplots()
ax2 = ax.twinx()
ax.grid(True)
ax.set_xlabel('Number of Clusters (k)')

ax.plot(K, inertias, label="Inertia", color="C0")
ax2.plot(K, distortions, label="Distortion", color="C1", linestyle="--")

ax.set_title('The Elbow Method using Distortion and Inertia')
ax.set_ylabel('Distortion')
ax2.set_ylabel('Inertia')
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper right')

In [ ]:
fig = plt.figure(figsize=(14, 6))
tag_to_marker = {'round': 'o', 'oval': '^'}
for idx, k in enumerate([2, 3], 1):
    kmeans = KMeans(n_clusters=k, random_state=42)
    y_kmeans = kmeans.fit_predict(fitting_array)

    ax = fig.add_subplot(1, 2, idx, projection='3d')

    for tag in df['groove_type'].unique():
        mask = df['groove_type'] == tag
        ax.scatter(
            df.loc[mask, 'tonnage'],
            df.loc[mask, 'normed_wear_area'],
            df.loc[mask, 'deformation_resistance'],
            c=y_kmeans[mask],
            cmap='viridis',
            marker=tag_to_marker[tag],
            edgecolor='k',
            label=f'groove_type: {tag}',
            s=60
        )

    ax.scatter(
        kmeans.cluster_centers_[:, 0],
        kmeans.cluster_centers_[:, 1],
        kmeans.cluster_centers_[:, 2],
        c='red', marker='x', s=200, label='Centroids'
    )

    ax.set_title(f'KMeans Clustering (k={k})')
    ax.set_xlabel('Tonnage')
    ax.set_ylabel('Normed Wear Area')
    ax.set_zlabel('Deformation Resistance')
    ax.legend(loc='upper left')

plt.tight_layout()
plt.show()